In [1]:
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from PIL import Image
import hsn_v1
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# path = 'img/02_glas_full'
path = 'img/02_glas_patch'

In [3]:
glas_paths = [str(path) for path in Path(path).rglob('*.png')]

In [4]:
glas_paths

['img/02_glas_patch/train_20_r1_c0.png',
 'img/02_glas_patch/testA_8_r0_c1.png',
 'img/02_glas_patch/train_25_r0_c0.png',
 'img/02_glas_patch/testA_57_r1_c1.png',
 'img/02_glas_patch/train_19_r0_c0.png',
 'img/02_glas_patch/train_63_r1_c1.png',
 'img/02_glas_patch/testA_35_r1_c1.png',
 'img/02_glas_patch/testA_40_r0_c1.png',
 'img/02_glas_patch/testA_23_r0_c1.png',
 'img/02_glas_patch/train_64_r1_c1.png',
 'img/02_glas_patch/train_51_r0_c1.png',
 'img/02_glas_patch/testA_27_r0_c0.png',
 'img/02_glas_patch/testB_7_r0_c0.png',
 'img/02_glas_patch/testA_10_r0_c1.png',
 'img/02_glas_patch/train_23_r0_c0.png',
 'img/02_glas_patch/train_40_r0_c1.png',
 'img/02_glas_patch/train_82_r1_c1.png',
 'img/02_glas_patch/train_66_r1_c0.png',
 'img/02_glas_patch/testA_16_r1_c1.png',
 'img/02_glas_patch/testA_46_r0_c1.png',
 'img/02_glas_patch/train_74_r1_c0.png',
 'img/02_glas_patch/testA_5_r1_c1.png',
 'img/02_glas_patch/testA_52_r0_c1.png',
 'img/02_glas_patch/testA_53_r0_c0.png',
 'img/02_glas_patch

In [5]:
glas_test = [file for file in glas_paths if 'test' in file]
glas_train = [file for file in glas_paths if 'train' in file]

In [6]:
X_train = []
for file in glas_train:
    img = Image.open(file)
    X_train.append(np.array(img))
X_train = np.array(X_train)

In [7]:
X_test = []
for file in glas_test:
    img = Image.open(file)
    X_test.append(np.array(img))
X_test = np.array(X_test)

In [8]:
X_train.shape, X_test.shape

((340, 224, 224, 3), (320, 224, 224, 3))

### Prepare labels

In [9]:
GO_INDEX = 48

In [10]:
y_train = np.zeros((X_train.shape[0], 51))
y_train[:,GO_INDEX] = 1

In [11]:
y_test = np.zeros((X_test.shape[0], 51))
y_test[:,GO_INDEX] = 1

In [12]:
# Validation set split 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train.shape, X_val.shape

((272, 224, 224, 3), (68, 224, 224, 3))

### Load model

In [13]:
MODEL_NAME = 'histonet_X1.7_clrdecay_5'
INPUT_NAME = '02_glas_full'
INPUT_MODE = 'patch'                    # {'patch', 'wsi'}
INPUT_SIZE = [224, 224]                 # [<int>, <int>] > 0
HTT_MODE = 'glas'                       # {'both', 'morph', 'func', 'glas'}
BATCH_SIZE = 1                          # int > 0
GT_MODE = 'on'                          # {'on', 'off'}
RUN_LEVEL = 3                           # {1: HTT confidence scores, 2: Grad-CAMs, 3: Segmentation masks}
SAVE_TYPES = [1, 1, 1, 1]               # {HTT confidence scores, Grad-CAMs, Segmentation masks, Summary images}
VERBOSITY = 'QUIET'                    # {'NORMAL', 'QUIET'}
# Settings for image set
IN_PX_RESOL = 0.620
OUT_PX_RESOL = 0.25 * 1088 / 224    # 1.21428571429
DOWNSAMPLE_FACTOR = OUT_PX_RESOL / IN_PX_RESOL

In [14]:
hsn = hsn_v1.HistoSegNetV1(params={'input_name': INPUT_NAME, 'input_size': INPUT_SIZE, 'input_mode': INPUT_MODE,
                                       'down_fac': DOWNSAMPLE_FACTOR, 'batch_size': BATCH_SIZE, 'htt_mode': HTT_MODE,
                                       'gt_mode': GT_MODE, 'run_level': RUN_LEVEL, 'save_types': SAVE_TYPES,
                                       'verbosity': VERBOSITY})

In [15]:
hsn.load_histonet(params={'model_name': MODEL_NAME}, pretrained=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loading pretrained weights : histonet_X1.7_clrdecay_5


In [16]:
histonet = hsn.hn
histonet.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 64)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 64)      256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
activation_2 (Activation)    (None, 224, 224, 64)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 224, 224, 64)      256       
__________

In [17]:
X_train.max(), X_train.min(), X_test.max(), X_test.min(), X_val.max(), X_val.min()

(255, 0, 255, 0, 255, 0)

In [18]:
X_train = histonet.normalize_image(X_train, is_glas=True)
X_test = histonet.normalize_image(X_test, is_glas=True)
X_val = histonet.normalize_image(X_val, is_glas=True)
X_train.max(), X_train.min(), X_test.max(), X_test.min(), X_val.max(), X_val.min()

(1.096684119538053,
 -3.4205767514322516,
 1.096684119538053,
 -3.4205767514322516,
 1.096684119538053,
 -3.4205767514322516)

### Train model

In [19]:
num_epochs = 30
batch_size = 8
steps_per_epoch = X_train.shape[0] // batch_size

In [20]:
model = histonet.model

In [21]:
model_chkpt = keras.callbacks.ModelCheckpoint(filepath='data/histonet_glas_holdout_ft.h5', monitor='val_loss', verbose=1,
                                             save_best_only=True, save_weights_only=True)

In [22]:
train_gen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
train_generator = train_gen.flow(X_train, y_train, batch_size=batch_size)

In [23]:
history = model.fit_generator(train_generator, 
                        epochs=num_epochs, 
                        verbose=1, 
                        shuffle=True, 
                        callbacks=[model_chkpt],
                        validation_data=(X_val, y_val),
                        steps_per_epoch=steps_per_epoch)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
34/34 [==============================] - 16s 476ms/step - loss: 0.0796 - binary_accuracy: 0.9941 - val_loss: 0.0818 - val_binary_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.08182, saving model to data/histonet_glas_holdout_ft.h5
Epoch 2/30
34/34 [==============================] - 12s 367ms/step - loss: 0.0814 - binary_accuracy: 1.0000 - val_loss: 0.0796 - val_binary_accuracy: 1.0000

Epoch 00002: val_loss improved from 0.08182 to 0.07959, saving model to data/histonet_glas_holdout_ft.h5
Epoch 3/30
34/34 [==============================] - 12s 366ms/step - loss: 0.0769 - binary_accuracy: 1.0000 - val_loss: 0.0745 - val_binary_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.07959 to 0.07448, saving model to data/histonet_glas_holdout_ft.h5
Epoch 4/30
34/34 [==============================] - 12s 361ms/step - loss: 0.0719 - binary_accuracy: 1.0000 - val_loss: 0.0696 - val_binary_accuracy: 1.0000

Epoch 000

In [24]:
from matplotlib import pyplot as plt

In [25]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label = 'val loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [26]:
history.history['val_loss']

[0.08182233265217613,
 0.07958917407428517,
 0.07447776724310483,
 0.06961257405140821,
 0.06514983229777392,
 0.060959199114757424,
 0.05717043797759449,
 0.05325975575867821,
 0.049625542014837265,
 0.046556481105439806,
 0.043494197156499416,
 0.04064963056760676,
 0.0379562167560353,
 0.03542325711425613,
 0.03300324476817075,
 0.030772096970502066,
 0.028779417714651895,
 0.026825738303801593,
 0.02535031012752477,
 0.023364323574830505,
 0.022099559578825447,
 0.02049830314867637,
 0.01927740521290723,
 0.017816952703630224,
 0.01675543309572865,
 0.01547833696445998,
 0.014436638749697629,
 0.013586875106043676,
 0.012689399532973766,
 0.0118160389144631]